verse-check-api/
├── app/
│   ├── main.py                # FastAPI entry point
│   ├── vector_store.py        # Qdrant setup and search logic
│   ├── embedding.py           # Hugging Face model loader
│   └── bible_loader.py        # Load & embed Bible verses
├── data/
│   └── bible.json             # KJV Bible data (in JSON format)
├── requirements.txt           # Python dependencies
└── README.md                  # Project overview

Here’s a simple, organized folder structure for your Verse Check API using FastAPI + Hugging Face + Qdrant, ready to paste into Visual Studio Code (VSC):

⸻

📁 Folder Structure

verse-check-api/
├── app/
│   ├── main.py                # FastAPI entry point
│   ├── vector_store.py        # Qdrant setup and search logic
│   ├── embedding.py           # Hugging Face model loader
│   └── bible_loader.py        # Load & embed Bible verses
├── data/
│   └── bible.json             # KJV Bible data (in JSON format)
├── requirements.txt           # Python dependencies
└── README.md                  # Project overview


⸻

✅ Step-by-Step Setup

1. Create the Folder

Open a terminal and run:

mkdir -p verse-check-api/app verse-check-api/data
cd verse-check-api

2. Install Python Dependencies

In verse-check-api/requirements.txt, paste:

fastapi
uvicorn
sentence-transformers
qdrant-client

Then install:

pip install -r requirements.txt


⸻

📄 File Templates (Copy/Paste These)

🔹 app/main.py

from fastapi import FastAPI
from pydantic import BaseModel
from app.embedding import get_model
from app.vector_store import get_client, search_verse

app = FastAPI()
model = get_model()
client = get_client()

class Query(BaseModel):
    quote: str

@app.post("/check")
def check_quote(query: Query):
    return search_verse(client, model, query.quote)


⸻

🔹 app/embedding.py

from sentence_transformers import SentenceTransformer

def get_model():
    return SentenceTransformer("all-MiniLM-L6-v2")


⸻

🔹 app/vector_store.py

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

def get_client():
    client = QdrantClient(":memory:")  # Use Qdrant Cloud here if you want persistence
    client.recreate_collection(
        collection_name="bible",
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )
    return client

def search_verse(client, model, quote):
    vector = model.encode(quote).tolist()
    search = client.search(
        collection_name="bible",
        query_vector=vector,
        limit=1,
        score_threshold=0.75
    )
    if search:
        m = search[0]
        return {
            "match": True,
            "score": m.score,
            "reference": f"{m.payload['book']} {m.payload['chapter']}:{m.payload['verse']}",
            "text": m.payload["text"]
        }
    return {"match": False, "message": "No verse found. Possibly a misattributed quote."}


⸻

🔹 app/bible_loader.py

(Only run once to populate Qdrant)

import json
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct

def load_and_embed():
    model = SentenceTransformer("all-MiniLM-L6-v2")
    client = QdrantClient(":memory:")

    with open("data/bible.json") as f:
        verses = json.load(f)

    points = []
    for i, verse in enumerate(verses):
        embedding = model.encode(verse["text"]).tolist()
        points.append(PointStruct(id=i, vector=embedding, payload=verse))

    client.upsert("bible", points=points)
    print("Loaded and embedded Bible verses.")

if __name__ == "__main__":
    load_and_embed()


⸻

🔹 data/bible.json

You can convert the KJV Bible to this format:

[
  {
    "book": "John",
    "chapter": 3,
    "verse": 16,
    "text": "For God so loved the world..."
  },
  ...
]

I can help you generate or clean this file if needed.

⸻

▶️ Run It

From the root folder:

uvicorn app.main:app --reload

Access the API at:
http://localhost:8000/docs

⸻